## 타이타닉 데이터 분석

* 다양한 머신러닝 알고리즘을 이용해서 교차검증 방식으로 모델을 훈련시키고 예측 정확도를 평가해 봄

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
titanic = pd.read_csv('csv/titanic.csv')
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,life,seat,port
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,S,live,1st,southampthon
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,S,live,1st,southampthon
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,S,dead,1st,southampthon
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,S,dead,1st,southampthon
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,S,dead,1st,southampthon


In [5]:
# 결측치 확인
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306 entries, 0 to 1305
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1306 non-null   int64  
 1   survived  1306 non-null   int64  
 2   name      1306 non-null   object 
 3   sex       1306 non-null   object 
 4   age       1306 non-null   float64
 5   sibsp     1306 non-null   int64  
 6   parch     1306 non-null   int64  
 7   ticket    1306 non-null   object 
 8   fare      1306 non-null   float64
 9   embarked  1306 non-null   object 
 10  life      1306 non-null   object 
 11  seat      1306 non-null   object 
 12  port      1306 non-null   object 
dtypes: float64(2), int64(4), object(7)
memory usage: 132.8+ KB


In [6]:
# 레이블 분포 확인
titanic.life.value_counts()

dead    808
live    498
Name: life, dtype: int64

In [8]:
# 여러특성들 중 좌석 분포 확인
titanic.seat.value_counts()

3rd    708
1st    321
2nd    277
Name: seat, dtype: int64

In [10]:
# 여러 특성들 중 성별 분포 확인
titanic.sex.value_counts()

male      842
female    464
Name: sex, dtype: int64

In [12]:
# 여러 특성들 중 승선위치 분포 확인
titanic.port.value_counts()

southampthon    913
cherbourg       270
qeenstown       123
Name: port, dtype: int64

In [13]:
# 데이터 분석시 문자형 값보다는 숫자형 값을 더 잘 인식함
# 문자형값 -> 숫자형값으로 변환하는 과정 필요
# 성별을 레이블인코딩으로 숫자형으로 변환 -> 파생변수

titanic['gender'] = titanic['sex'].apply(lambda x: 0 if x == 'female' else 1)

titanic.iloc[:, [3, 13]].head(5)

# titanic.loc[:, ['sex', 'gender']].head(5)



,sex,gender
0,female,0
1,male,1
2,female,0
3,male,1
4,female,0


In [15]:
# 승선위치를 레이블인코딩으로 숫자형으로 변환 -> 파생변수
titanic['harbor'] = titanic['embarked'].apply(lambda x: 0 if x == 'C' else (1 if x == 'S' else 2))
titanic.iloc[:, [9,14]].head(5)

,embarked,harbor
0,S,1
1,S,1
2,S,1
3,S,1
4,S,1


In [16]:
# 분석에 필요한 컬럼을 뽑아 특성/레이블을 만듦
data = titanic.iloc[:, [0, 4, 5, 6, 8, 13, 14]]
target = titanic.survived

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [18]:
# 훈련/평가 데이터 분할
Xtrain, Xtest, ytrain, ytest = train_test_split(data, target, train_size = 0.7, random_state = 2111041110)

In [19]:
# 알고리즘 적용
# 의사결정나무
dtclf = DecisionTreeClassifier()
dtclf.fit(Xtrain, ytrain)
pred = dtclf.predict(Xtest)
accuracy_score(ytest, pred) # 0.77

0.6301020408163265

In [20]:
# 로지스틱 회귀
lrclf = LogisticRegression()
lrclf.fit(Xtrain, ytrain)
pred = lrclf.predict(Xtest)
accuracy_score(ytest, pred)

/home/hadoop/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7193877551020408

In [22]:
# 랜덤포레스트
rfclf = RandomForestClassifier()
rfclf.fit(Xtrain, ytrain)
pred = rfclf .predict(Xtest)
accuracy_score(ytest, pred) # 0.78

0.6530612244897959

In [25]:
# 교차검증 1
dtclf = DecisionTreeClassifier(max_depth = 3)
scores = cross_val_score(dtclf, data, target, cv=10, scoring='accuracy')
np.mean(scores)

0.63936582501468

In [26]:
# 머신러닝 모델 평가
# 정확도만으로 모델의 성능을 평가하는 것이 옳은것인가?
titanic.life.value_counts()

dead    808
live    498
Name: life, dtype: int64

In [33]:
# 성별에 따른 생존 여부
titanic.groupby(['sex','life'])['life'].count()


# 여성의 생존율이 남성의 생존율 보다 높기 때문에 간단한 조건문만으로 모델을 만들수도 있다.
# 입력값 : 여성 -> 생존 , 남성 -> 사망

sex     life
female  dead    127
        live    337
male    dead    681
        live    161
Name: life, dtype: int64

## 머신러닝 모델 평가
* 일반적으로 머신러닝은
   + 데이터가공/변환(전처리)
   + 모델 학습/예측
   + 평가의 과정을 거침
* 앞의 타이타닉 분석에서 모델의 평가는 정확도만 사용함
* 한편, 머신러닝의 예측성능의 평가방법은
  + 회귀 : R^2, MSE평균제곱오차
  + 분류 : 오차행렬, 정밀도/재현율, ROC, AUC, F1스코어,
          크로스엔트로피, 최대우도

## 정확도의 함정
* 탐색적 분석 시행시 성별 기준 생존비율은 여성이 더 높음
* 따라서, 굳이 ML알고리즘을 적용하지 않아도 성별이 여성일 경우 생존, 남성일 경우 사망이라고 예측해도 크게 무리가 없음  
* 즉, 단순히 성별 조건 하나만 적용해도 별거 아닌 알고리즘으로도 높은 정확도가 나타나는 상황 발생

In [38]:
# BaseEstimator 클래스를 상속받아 가짜분류기 생성
from sklearn.base import BaseEstimator
class DummyClassifier(BaseEstimator):
    # 아무것도 학습하지 않는 fit 메서드 정의
    def fit(self, X, y=None):
        pass 
# 성별이 1(남자)이면 사망(0)
# 성별이 0(여자)이면 생존(1)이라고
# 예측하는 predict메서드 정의
    def predict(self, X):
        # 입력데이터 크기만큼 0으로 채워진 1차원 행렬 생성
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
           # 성별이 여성이면 무조건 생존이라 예측
           if X['sex'].iloc[i] != 1:
              pred[i] = 1
        return pred

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [45]:
titanic = pd.read_csv('csv/titanic.csv')
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,life,seat,port
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,S,live,1st,southampthon
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,S,live,1st,southampthon
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,S,dead,1st,southampthon
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,S,dead,1st,southampthon
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,S,dead,1st,southampthon


In [41]:
titanic['sex'] = titanic['sex'].apply(lambda x:0 if x=='female' else 1)

In [42]:
data = titanic.iloc[:, [0,3,4,5,6,8]]
target = titanic.survived

In [43]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data,target,train_size=0.7, random_state=2111041205)

In [44]:
myclf = DummyClassifier()
myclf.fit(Xtrain, ytrain)
pred = myclf.predict(Xtest)
accuracy_score(pred, ytest)

0.798469387755102